### 图像轮廓
- 在轮廓检测前必须进行二值化处理
- cv2.findContours(img,mode,method)
1) mode只记住最后一个RETR_TREE, 其它结果都可从中调用
2) method即轮廓的逼近方法<br>
CHAIN_APPROX_NONE: 输出多边形<br>
CHAIN_APPROX_SIMPLE: 只保留终点部分


In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [30]:
def cv_show(img, title='Image'):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [54]:
img= cv2.imread('Contours.png', cv2.IMREAD_GRAYSCALE)

In [58]:
img_color= cv2.imread('Contours.png')

In [14]:
thresh, ret= cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

In [49]:
contours, hierarchy= cv2.findContours(ret, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

In [59]:
# -1 means draw all contours, 2 means thickness of the contour lines
res= cv2.drawContours(img_color, contours, -1, (0, 0, 255), 2)
cv_show(res, "res")